In [20]:
import torch
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import numpy as np

from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import qUpperConfidenceBound
from botorch.optim import sequential_optimize, joint_optimize
from botorch.utils.sampling import manual_seed

In [21]:
train_dataset = dsets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = dsets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

In [22]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)

In [23]:
batch_size = 100
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [24]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs

In [25]:
class MultiLayerNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_units_dim, output_dim):
        super(MultiLayerNN, self).__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden_units_dim),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(hidden_units_dim),
            torch.nn.Linear(hidden_units_dim, hidden_units_dim),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(hidden_units_dim),
            torch.nn.Linear(hidden_units_dim, hidden_units_dim),
        )
        for m in self.model:
            if isinstance(m, torch.nn.Linear):
                torch.nn.init.kaiming_normal_(m.weight)
                torch.nn.init.constant_(m.bias, 0)
        
    def forward(self, x):
        return self.model(x)

In [26]:
batch_size = 100
epochs = 30
input_dim = 784
output_dim = 10
lr_rate = 0.05
z_dim = 5
z_length = 50
len_train_dataset = len(train_loader.dataset)

In [27]:
model = MultiLayerNN(input_dim, 1000, output_dim).cuda()
alt_model = MultiLayerNN(input_dim, 1000, output_dim).cuda()
criterion = torch.nn.CrossEntropyLoss().cuda()

In [33]:
def _seperate_w(w, model):
    w_reshape_list = []
    num_params = 0
    for param in model.parameters():
        w_reshape = w[num_params:num_params + param.shape.numel()].reshape_as(param)
        w_reshape_list.append(w_reshape)
        num_params += param.shape.numel()
    
    return w_reshape_list

def _get_w_set(w, model):
    w_set = [_seperate_w(w[:, i].cuda(), model) for i in range(w.shape[1])]
    return w_set

def _get_w(A, z, c, d):
    sig = torch.nn.Sigmoid()
    lr_rate = z.transpose(0, 1)[:, 0]
    lr_rate = (d - 1) * lr_rate + 1
    h = 2 * sig(c * A.mm(z[1:, :]))
    return lr_rate * h.clamp(min=1.0)

def _get_alt_model_loss(x, y, model, alt_model, criterion, w_set):
    alt_losses = []
    
    for w_reshape in w_set:
        for param, alt_param, w in zip(model.parameters(), alt_model.parameters(), w_reshape):
            #print(param.shape, alt_param.shape, w.shape, param.grad.shape)
            alt_param.data = param.data - lr_rate * w.data * param.grad.data
            #alt_param.data = param.data - lr_rate * (1) * param.grad.data
        
        alt_outputs = alt_model(x)
        alt_loss = criterion(alt_outputs, y)

        alt_losses.append(alt_loss)

    return torch.tensor(alt_losses)

class AdaptiveGPGradient:
    def __init__(self, model, z_dim, z_length, update_weight_step, fit_gp_step):
        w_dim = sum([param.shape.numel() for param in model.parameters()])
        
        self.A = torch.randn([w_dim, z_dim - 1])
        self.Z = torch.rand([z_dim, z_length])
        
        self.model = model
        
        self.step = 0
        self.update_weight_step = update_weight_step
        self.fit_gp_step = fit_gp_step
        
        self.c = 1.0
        self.d = 10.0
        
        self.z_length = z_length
        self.z_dim = z_dim
        
        self.W = _get_w(self.A, self.Z, self.c, self.d)
        self.w_set = _get_w_set(self.W, model)
        self.V = torch.zeros(self.z_length)
        
        self.best_z = torch.zeros([z_dim ,1])
        self.best_w = _get_w_set(_get_w(self.A, self.best_z, self.c, self.d), self.model)[0]
        #self.fit_gp_step = fit_gp_step
    
    def gather_V(self, images, labels, model, criterion):
        with torch.no_grad():
            V = _get_alt_model_loss(images, labels, model, self.model, criterion, self.w_set)
            best_V = _get_alt_model_loss(images, labels, model, self.model, criterion, [self.best_w])
            self.V += (best_V - V) / self.update_weight_step
            #print(self.V)
        
    def update_Z(self):
        if self.step < self.update_weight_step:
            self.step += 1
            return
        else:
            self.step = 0
            print(self.V)
        
        train_X = self.Z.transpose(0, 1).cpu().data
        train_X = torch.cat([train_X, self.best_z.reshape([1, -1])], 0)
        
        train_Y = self.V.reshape([-1, 1]).data
        train_Y = torch.cat([train_Y, torch.zeros([1, 1])], 0)
        train_Y = (train_Y - train_Y.min()) / (train_Y.max() - train_Y.min())
        
        with manual_seed(0):
            gp = SingleTaskGP(train_X, train_Y)
            mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
            fit_gpytorch_model(mll)

            qUCB = qUpperConfidenceBound(gp, beta=0.1)

            z_dim = self.Z.shape[0]
            bounds = torch.stack([torch.zeros(z_dim), torch.ones(z_dim)])

            # Get best_z
            self.best_z, _ = joint_optimize(
                qUCB, bounds=bounds, q=1, num_restarts=20, raw_samples=512,
            )
        self.best_z.transpose_(0, 1)
        self.best_w = _get_w_set(_get_w(self.A, self.best_z, self.c, self.d), self.model)[0]
        
        # Get candidates
        arg_sorted = train_Y.reshape([-1]).argsort()
        arg_sorted = arg_sorted[1:]
        self.Z = train_X[arg_sorted].transpose(0, 1)
        W = _get_w(self.A, self.Z, self.c, self.d)
        self.w_set = _get_w_set(W, model)
        
        # Reset V
        self.V = torch.zeros(self.z_length)
        print(self.Z.transpose(0, 1))
        
test = AdaptiveGPGradient(alt_model, z_dim, z_length, 100, 1)

In [34]:
def show_params(model):
    for param in model.parameters():
        print(param.data)

In [35]:
results = []
adaptive_gp_gradient = AdaptiveGPGradient(alt_model, z_dim, z_length, 100, 1)

for epoch in range(epochs):
    avg_loss = 0.
    
    increments = torch.zeros(z_length)
    
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28 * 28)).cuda()
        labels = Variable(labels).cuda()
        
        outputs = model(images)
        loss = criterion(outputs, labels)

        model.zero_grad()
        loss.backward()
        
        avg_loss += loss / (len_train_dataset / batch_size)
        
        with torch.no_grad():
            adaptive_gp_gradient.gather_V(images, labels, model, criterion)
            for w, param in zip(adaptive_gp_gradient.best_w, model.parameters()):
                param.data -= lr_rate * w.data * param.grad.data
        
        adaptive_gp_gradient.update_Z()
    results.append(avg_loss)
    
    print(epoch, avg_loss.data)
    

tensor([ 0.0327,  0.0441,  0.0080, -0.0181,  0.0435,  0.0460,  0.0428,  0.0461,
        -0.0023,  0.0239,  0.0121,  0.0094,  0.0287,  0.0339,  0.0460,  0.0443,
         0.0272,  0.0452,  0.0461,  0.0460,  0.0127,  0.0141,  0.0197, -0.0068,
         0.0163,  0.0385,  0.0234,  0.0246,  0.0162,  0.0418,  0.0429,  0.0242,
         0.0198,  0.0461,  0.0301, -0.0192,  0.0452,  0.0460,  0.0410,  0.0263,
         0.0410,  0.0372,  0.0447, -0.0080,  0.0461,  0.0364,  0.0435,  0.0140,
         0.0352,  0.0184])
tensor([[9.8072e-01, 8.4359e-02, 8.3079e-01, 5.8914e-01, 9.6839e-01],
        [9.2766e-01, 6.3870e-01, 7.9743e-01, 1.9470e-01, 8.6792e-01],
        [9.4674e-01, 6.9452e-01, 5.6195e-01, 6.6222e-01, 1.4673e-01],
        [9.6476e-01, 3.9471e-01, 5.9279e-01, 1.8843e-01, 1.7079e-01],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [9.0318e-01, 4.4584e-01, 3.4564e-01, 2.2082e-01, 3.1590e-01],
        [8.0913e-04, 5.1878e-01, 1.2082e-01, 6.7800e-01, 4.9150e-01],
   

tensor([[9.4674e-01, 6.9452e-01, 5.6195e-01, 6.6222e-01, 1.4673e-01],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [9.6476e-01, 3.9471e-01, 5.9279e-01, 1.8843e-01, 1.7079e-01],
        [8.0913e-04, 5.1878e-01, 1.2082e-01, 6.7800e-01, 4.9150e-01],
        [9.0318e-01, 4.4584e-01, 3.4564e-01, 2.2082e-01, 3.1590e-01],
        [1.8829e-04, 8.9104e-01, 9.1278e-01, 5.6687e-01, 7.4393e-01],
        [7.8043e-01, 9.7217e-01, 5.6940e-01, 7.9250e-01, 5.9985e-01],
        [7.9057e-01, 9.1008e-02, 7.5588e-01, 9.6191e-01, 3.4282e-01],
        [7.7018e-01, 8.7775e-01, 9.7685e-01, 4.5194e-01, 5.4957e-01],
        [8.0248e-01, 3.9792e-01, 5.9638e-01, 3.8748e-01, 5.0339e-01],
        [7.5308e-01, 9.5779e-01, 4.0928e-01, 8.1684e-01, 7.1736e-01],
        [7.6203e-01, 5.7109e-01, 6.0378e-02, 5.3277e-01, 8.8945e-01],
        [7.5247e-01, 1.6485e-01, 8.1469e-01, 7.6826e-01, 3.3236e-01],
        [7.5344e-01, 1.9061e-02, 5.7244e-01, 7.0895e-01, 7.2229e-01],
        [3.8496e-02,

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [9.0318e-01, 4.4584e-01, 3.4564e-01, 2.2082e-01, 3.1590e-01],
        [7.8043e-01, 9.7217e-01, 5.6940e-01, 7.9250e-01, 5.9985e-01],
        [7.9057e-01, 9.1008e-02, 7.5588e-01, 9.6191e-01, 3.4282e-01],
        [8.0913e-04, 5.1878e-01, 1.2082e-01, 6.7800e-01, 4.9150e-01],
        [7.7018e-01, 8.7775e-01, 9.7685e-01, 4.5194e-01, 5.4957e-01],
        [8.0248e-01, 3.9792e-01, 5.9638e-01, 3.8748e-01, 5.0339e-01],
        [7.5308e-01, 9.5779e-01, 4.0928e-01, 8.1684e-01, 7.1736e-01],
        [1.8829e-04, 8.9104e-01, 9.1278e-01, 5.6687e-01, 7.4393e-01],
        [7.6203e-01, 5.7109e-01, 6.0378e-02, 5.3277e-01, 8.8945e-01],
        [7.5247e-01, 1.6485e-01, 8.1469e-01, 7.6826e-01, 3.3236e-01],
        [7.5344e-01, 1.9061e-02, 5.7244e-01, 7.0895e-01, 7.2229e-01],
        [6.9737e-01, 9.8295e-01, 8.0963e-01, 4.2114e-01, 3.7545e-01],
        [6.8151e-01, 8.6364e-01, 8.6325e-01, 7.8081e-01, 6.4854e-01],
        [7.2126e-01,

tensor([[1.8829e-04, 8.9104e-01, 9.1278e-01, 5.6687e-01, 7.4393e-01],
        [9.0318e-01, 4.4584e-01, 3.4564e-01, 2.2082e-01, 3.1590e-01],
        [7.8043e-01, 9.7217e-01, 5.6940e-01, 7.9250e-01, 5.9985e-01],
        [7.9057e-01, 9.1008e-02, 7.5588e-01, 9.6191e-01, 3.4282e-01],
        [7.7018e-01, 8.7775e-01, 9.7685e-01, 4.5194e-01, 5.4957e-01],
        [8.0248e-01, 3.9792e-01, 5.9638e-01, 3.8748e-01, 5.0339e-01],
        [7.5308e-01, 9.5779e-01, 4.0928e-01, 8.1684e-01, 7.1736e-01],
        [3.8496e-02, 6.4685e-01, 5.1635e-01, 1.8405e-01, 4.4886e-01],
        [7.6203e-01, 5.7109e-01, 6.0378e-02, 5.3277e-01, 8.8945e-01],
        [7.5247e-01, 1.6485e-01, 8.1469e-01, 7.6826e-01, 3.3236e-01],
        [7.5344e-01, 1.9061e-02, 5.7244e-01, 7.0895e-01, 7.2229e-01],
        [6.9737e-01, 9.8295e-01, 8.0963e-01, 4.2114e-01, 3.7545e-01],
        [6.8151e-01, 8.6364e-01, 8.6325e-01, 7.8081e-01, 6.4854e-01],
        [7.2126e-01, 2.8973e-02, 2.1017e-01, 9.9767e-01, 2.7731e-01],
        [6.4849e-02,

tensor([-1.2800e-02, -6.8778e-03, -1.2327e-02, -1.2229e-02, -1.1302e-02,
        -1.1239e-02, -4.8042e-03, -1.0315e-02, -9.8989e-03, -9.8045e-03,
        -8.1274e-03, -7.9576e-03, -7.8454e-03, -7.1615e-03, -2.1962e-03,
        -6.7909e-03, -5.5490e-03, -9.0458e-04, -4.5171e-03, -4.0780e-03,
        -3.9878e-04, -3.5365e-03, -1.1725e-04, -7.6184e-05, -2.6798e-05,
        -2.7801e-03,  1.7454e-04,  2.1939e-04, -2.2661e-03,  2.2361e-04,
         2.3182e-04,  3.0656e-04,  3.5993e-04, -1.3507e-03,  4.1317e-04,
         4.1741e-04,  4.2154e-04, -1.0892e-03,  4.1500e-04,  4.1329e-04,
         4.0069e-04,  3.9623e-04,  3.9691e-04, -7.2795e-04,  3.8752e-04,
         3.5744e-04,  3.5406e-04, -5.1221e-04, -4.9577e-04, -8.3137e-05])
tensor([[0.7906, 0.0910, 0.7559, 0.9619, 0.3428],
        [0.7702, 0.8778, 0.9769, 0.4519, 0.5496],
        [0.8025, 0.3979, 0.5964, 0.3875, 0.5034],
        [0.7531, 0.9578, 0.4093, 0.8168, 0.7174],
        [0.7620, 0.5711, 0.0604, 0.5328, 0.8895],
        [0.7525, 0.

...

tensor([ 4.4185e-06,  4.2025e-06, -3.3796e-05, -3.2980e-05, -3.2679e-05,
         3.9899e-06, -3.0694e-05, -2.9823e-05, -2.9589e-05, -2.9285e-05,
        -2.8324e-05, -2.8337e-05, -2.7288e-05, -2.7291e-05, -2.6967e-05,
        -2.6579e-05, -2.6629e-05, -2.6426e-05, -2.4991e-05, -2.5082e-05,
        -2.4053e-05, -2.2689e-05, -2.2628e-05, -2.2435e-05, -2.2289e-05,
        -2.2219e-05, -2.2183e-05, -2.1958e-05, -2.1883e-05, -2.1563e-05,
        -2.0700e-05, -2.0026e-05, -1.9860e-05, -1.9478e-05, -1.9412e-05,
        -1.8817e-05, -1.8666e-05, -1.7927e-05, -1.7551e-05, -1.6926e-05,
        -1.3500e-05, -1.3310e-05,  2.6916e-06,  2.6310e-06, -6.0956e-06,
        -4.8693e-06, -4.7811e-06, -3.9648e-06,  1.0592e-06, -1.0656e-06])
tensor([[0.4258, 0.2211, 0.2783, 0.7080, 0.0939],
        [0.4046, 0.9551, 0.1543, 0.4483, 0.4858],
        [0.4421, 0.4772, 0.4697, 0.3639, 0.0000],
        [0.4267, 0.9305, 0.0000, 0.2937, 0.4693],
        [0.4360, 0.6397, 0.3477, 0.5841, 0.1752],
        [0.4005, 0.

tensor([ 1.6606e-05,  1.6796e-05,  1.6874e-05,  1.6667e-05,  1.5864e-05,
         1.5863e-05,  1.4589e-05,  1.3631e-05,  1.2674e-05,  9.7356e-06,
         8.7702e-06,  8.9423e-06,  7.2942e-06, -2.5765e-05, -2.1999e-05,
        -2.3234e-05, -2.1310e-05, -2.1733e-05, -2.0104e-05, -1.8752e-05,
        -2.0242e-05, -1.8819e-05, -1.8269e-05, -1.7867e-05, -1.7852e-05,
        -1.7597e-05, -1.5852e-05, -1.5771e-05, -1.2700e-05, -1.2389e-05,
        -1.2671e-05, -1.4534e-05, -1.1771e-05, -1.0184e-05, -1.2383e-05,
        -1.1415e-05, -1.1686e-05, -9.4063e-06, -1.2261e-05, -8.6277e-06,
        -1.2641e-06, -9.6817e-07, -1.2082e-05, -9.1159e-06, -7.5727e-06,
        -8.5065e-06, -7.8640e-06, -6.6521e-06, -6.2108e-06, -5.5202e-06])
tensor([[0.4421, 0.4772, 0.4697, 0.3639, 0.0000],
        [0.4267, 0.9305, 0.0000, 0.2937, 0.4693],
        [0.4360, 0.6397, 0.3477, 0.5841, 0.1752],
        [0.4005, 0.8885, 0.3525, 1.0000, 1.0000],
        [0.5371, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4550, 0.

tensor([-8.2255e-04, -7.9289e-04, -7.0126e-04, -6.4469e-04, -6.2223e-04,
        -5.1187e-04, -5.1601e-04, -4.3012e-04, -3.8329e-04, -3.4082e-04,
        -2.5089e-04, -2.2630e-04, -2.2882e-04, -1.9746e-04, -8.6450e-05,
        -7.5236e-05, -7.1070e-05, -3.6136e-05, -3.9863e-05, -2.5527e-05,
        -2.6962e-05, -2.1207e-05, -3.1574e-05, -2.0341e-05, -7.3592e-06,
        -7.2546e-06, -1.7894e-05, -2.2431e-05, -9.6621e-06, -1.0587e-05,
        -1.5141e-05,  1.3742e-06, -7.8106e-06, -9.3263e-06, -2.7641e-06,
        -7.8495e-06, -4.0598e-07,  1.7614e-07, -3.4647e-06, -1.6849e-06,
         1.4467e-07, -2.8338e-06, -6.0758e-07, -9.2327e-06,  1.1440e-06,
         1.2929e-06, -3.7926e-06,  6.4697e-07, -1.0027e-05, -3.1032e-07])
tensor([[0.8025, 0.6010, 0.2896, 0.6524, 1.0000],
        [0.7790, 0.7120, 0.3009, 0.7243, 0.8625],
        [0.7734, 0.6049, 0.3308, 0.5838, 0.8642],
        [0.7552, 0.6083, 0.1848, 0.7343, 1.0000],
        [0.7407, 0.6918, 0.2111, 0.4543, 0.8197],
        [0.7236, 0.

tensor([-1.4962e-05, -1.4846e-05, -1.3518e-05, -1.3260e-05, -1.2088e-05,
        -1.2045e-05, -1.2248e-05, -1.2237e-05, -1.1726e-05, -1.1219e-05,
        -1.1264e-05, -9.8676e-06, -9.7198e-06, -8.8937e-06, -7.2898e-06,
        -7.4073e-06, -7.2535e-06, -6.8129e-06, -6.8929e-06, -7.0373e-06,
        -6.6760e-06, -6.5881e-06, -6.0030e-06, -5.4656e-06, -4.9218e-06,
        -4.6781e-06, -4.1594e-06, -4.3288e-06, -3.7992e-06, -3.8514e-06,
        -2.5525e-05, -2.9075e-06, -2.9971e-06, -2.1420e-06, -1.9275e-05,
         2.2612e-07,  2.5082e-07,  6.5250e-07, -1.5584e-05, -1.4413e-05,
        -1.0180e-05, -8.1800e-06, -8.2370e-06,  2.1132e-06, -4.0653e-06,
         1.7765e-06,  1.8199e-06,  1.3234e-06, -2.4315e-06, -8.5039e-07])
tensor([[0.7790, 0.7120, 0.3009, 0.7243, 0.8625],
        [0.7734, 0.6049, 0.3308, 0.5838, 0.8642],
        [0.4360, 0.6397, 0.3477, 0.5841, 0.1752],
        [0.4005, 0.8885, 0.3525, 1.0000, 1.0000],
        [0.7552, 0.6083, 0.1848, 0.7343, 1.0000],
        [0.4550, 0.

tensor([-0.0012, -0.0011, -0.0011, -0.0010, -0.0010, -0.0010, -0.0009, -0.0008,
        -0.0008, -0.0007, -0.0007, -0.0006, -0.0006, -0.0006, -0.0006, -0.0004,
        -0.0004, -0.0004, -0.0004, -0.0003, -0.0004, -0.0003, -0.0003, -0.0003,
        -0.0003, -0.0003, -0.0003, -0.0003, -0.0003, -0.0003, -0.0003, -0.0003,
        -0.0003, -0.0003, -0.0003, -0.0003, -0.0003, -0.0003, -0.0002, -0.0002,
        -0.0002, -0.0002, -0.0002, -0.0002, -0.0002, -0.0002, -0.0002, -0.0002,
        -0.0002, -0.0002])
tensor([[0.7734, 0.6049, 0.3308, 0.5838, 0.8642],
        [0.7552, 0.6083, 0.1848, 0.7343, 1.0000],
        [0.7466, 0.6601, 0.2280, 0.6872, 0.7655],
        [0.7407, 0.6918, 0.2111, 0.4543, 0.8197],
        [0.7236, 0.6363, 0.3522, 0.6055, 1.0000],
        [0.7134, 0.4117, 0.3707, 0.6472, 0.7746],
        [0.7100, 0.2754, 0.2732, 0.3822, 0.8524],
        [0.6933, 0.6444, 0.2677, 0.3355, 0.6566],
        [0.6790, 0.5314, 0.1074, 0.5342, 0.6827],
        [0.6471, 0.6901, 0.0000, 0.3358, 0.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(results)

In [ ]:
correct = 0.
total = 0.

for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28)).cuda()
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct+= (predicted.cpu() == labels).sum()

accuracy = 100 * correct.float() / total

In [ ]:
accuracy